# Import packages

In [ ]:
import sys

import h5py
import numpy as np
from skimage import transform as tf

sys.path.insert(0, '/dls_sw/i14/scripts/i14_python/xanes_utility')
# TODO: This is currently a workaround to not having a system-wide conda env
# with a compatible version of Hyperspy installed in it, and it's dependent on a
# local clone of the git repo which can be changed at any time, so it's not a
# robust solution...
sys.path.insert(0, '/dls_sw/i14/scripts/i14_python/tomo_utility/hyperspy')
import hyperspy.api as hs
from mutual import estimate_shift2D_mutual

# Load the combined data

In [ ]:

# Define input data file
in_file = ''

with h5py.File(in_file, 'r') as f:
    data = f['/entry/tomo_entry/data/data'][:, :, :]
    rot_angles = f['/entry/tomo_entry/data/rotation_angle'][:]
    im_key = f['/entry/tomo_entry/instrument/detector/image_key'][:]

# Calculate shifts

In [ ]:

# Create Signal2D objct to hold the data
signal = hs.signals.Signal2D(data)

# Get the shifts that are needed to align the data
reference = 'current'
roi = [] # left, right, top, bottom
shifts, max_vals = \
    estimate_shift2D_mutual(signal, reference=reference, roi=roi)

# OPTIONAL: Save shifts to a file
#shifts_file_path = ''
#np.save(shifts_file_path, shifts)

# Apply shifts to data

In [ ]:
shifted_data = np.zeros(shape=data.shape)

# Loop through all projections and shift them accordingly
for i in range(data.shape[0]):
    x_shift = shifts[i][1]
    y_shift = shifts[i][0]
    transformation = \
        tf.SimilarityTransform(translation=(x_shift, y_shift))
    transformed_image = tf.warp(data[i], transformation, order=0)
    shifted_data[i,:,:] = transformed_image

# Save aligned data

In [ ]:

# Define path for output file
out_file_path = ''

with h5py.File(out_file_path, 'a') as f_out:
    f_out.create_dataset('/entry/tomo_entry/data/data', data=shifted_data)
    f_out.create_dataset('/entry/tomo_entry/data/rotation_angle',
                            data=rot_angles)
    f_out.create_dataset('/entry/tomo_entry/instrument/detector/image_key',
                            data=im_key)